In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Text
from dfply import *

%matplotlib inline

In [2]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

2021-10-19 19:44:22.565014: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Retrieval - matrix factorisation tower

In [3]:
ratings

<PrefetchDataset shapes: {bucketized_user_age: (), movie_genres: (None,), movie_id: (), movie_title: (), raw_user_age: (), timestamp: (), user_gender: (), user_id: (), user_occupation_label: (), user_occupation_text: (), user_rating: (), user_zip_code: ()}, types: {bucketized_user_age: tf.float32, movie_genres: tf.int64, movie_id: tf.string, movie_title: tf.string, raw_user_age: tf.float32, timestamp: tf.int64, user_gender: tf.bool, user_id: tf.string, user_occupation_label: tf.int64, user_occupation_text: tf.string, user_rating: tf.float32, user_zip_code: tf.string}>

In [4]:
ratings_reduced = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_occupation":x["user_occupation_text"],
})
movies_reduced = movies.map(lambda x: x["movie_title"])

In [5]:
ratings_reduced

<MapDataset shapes: {movie_title: (), user_id: (), user_occupation: ()}, types: {movie_title: tf.string, user_id: tf.string, user_occupation: tf.string}>

In [6]:
tf.random.set_seed(42)
shuffled = ratings_reduced.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [7]:
movie_titles = movies_reduced.batch(1_000)
user_ids = ratings_reduced.batch(1_000_000).map(lambda x: x["user_id"])
user_occupation = ratings_reduced.batch(1_000_000).map(lambda x: x["user_occupation"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
unique_user_occupations = np.unique(np.concatenate(list(user_occupation)))

unique_movie_titles[:10]

2021-10-19 19:44:22.857028: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [8]:
embedding_dimension = 32

In [9]:
user_model = tf.keras.Sequential([
    #user id feature
    tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [10]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [11]:
import tensorflow_recommenders as tfrs
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies_reduced.batch(128).map(movie_model)
)

In [12]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [13]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [14]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [15]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [16]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 10s 767ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0013 - factorized_top_k/top_5_categorical_accuracy: 0.0098 - factorized_top_k/top_10_categorical_accuracy: 0.0205 - factorized_top_k/top_50_categorical_accuracy: 0.1005 - factorized_top_k/top_100_categorical_accuracy: 0.1774 - loss: 69885.1129 - regularization_loss: 0.0000e+00 - total_loss: 69885.1129
Epoch 2/3
10/10 [==============================] - 7s 700ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0028 - factorized_top_k/top_5_categorical_accuracy: 0.0188 - factorized_top_k/top_10_categorical_accuracy: 0.0380 - factorized_top_k/top_50_categorical_accuracy: 0.1696 - factorized_top_k/top_100_categorical_accuracy: 0.2931 - loss: 67523.3707 - regularization_loss: 0.0000e+00 - total_loss: 67523.3707
Epoch 3/3
10/10 [==============================] - 7s 663ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0035 - factorized_top_k/top_5_categorical_accuracy:

In [17]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 2s 303ms/step - factorized_top_k/top_1_categorical_accuracy: 9.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0096 - factorized_top_k/top_10_categorical_accuracy: 0.0224 - factorized_top_k/top_50_categorical_accuracy: 0.1250 - factorized_top_k/top_100_categorical_accuracy: 0.2327 - loss: 31079.0635 - regularization_loss: 0.0000e+00 - total_loss: 31079.0635


{'factorized_top_k/top_1_categorical_accuracy': 0.0008999999845400453,
 'factorized_top_k/top_5_categorical_accuracy': 0.009600000455975533,
 'factorized_top_k/top_10_categorical_accuracy': 0.022350000217556953,
 'factorized_top_k/top_50_categorical_accuracy': 0.125,
 'factorized_top_k/top_100_categorical_accuracy': 0.23270000517368317,
 'loss': 28244.771484375,
 'regularization_loss': 0,
 'total_loss': 28244.771484375}

In [20]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies_reduced.batch(100), movies_reduced.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)'
 b'Father of the Bride Part II (1995)' b'Rudy (1993)']


Investgate adding user occupation

In [27]:
class UserModel(tf.keras.Model):

  def __init__(self, use_occupation):
    super().__init__()

    self._use_occupation = use_occupation

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_occupation:
        self.user_embedding2 = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_occupations, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_occupations) + 1, 32),
    ])


  def call(self, inputs):
    if not self._use_occupation:
      return self.user_embedding(inputs["user_id"])

    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.user_embedding2(inputs["user_occupation"])
    ],1)

In [22]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])

  def call(self, titles):
    return self.title_embedding(titles)

In [23]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, use_occupations):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(use_occupations),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      MovieModel(),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies_reduced.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "user_occupation": features["user_occupation"],
    })
    movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(query_embeddings, movie_embeddings)

In [24]:
tf.random.set_seed(42)
shuffled = ratings_reduced.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [25]:
model = MovielensModel(use_occupations=False)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


40/40 [==============================] - 8s 168ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0133 - factorized_top_k/top_5_categorical_accuracy: 0.0263 - factorized_top_k/top_10_categorical_accuracy: 0.0362 - factorized_top_k/top_50_categorical_accuracy: 0.0937 - factorized_top_k/top_100_categorical_accuracy: 0.1547 - loss: 14662.5915 - regularization_loss: 0.0000e+00 - total_loss: 14662.5915
Epoch 2/3
40/40 [==============================] - 8s 177ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0034 - factorized_top_k/top_5_categorical_accuracy: 0.0144 - factorized_top_k/top_10_categorical_accuracy: 0.0272 - factorized_top_k/top_50_categorical_accuracy: 0.1142 - factorized_top_k/top_100_categorical_accuracy: 0.2136 - loss: 14197.1637 - regularization_loss: 0.0000e+00 - total_loss: 14197.1637
Epoch 3/3
40/40 [==============================] - 8s 177ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0018 - factorized_top_k/top_5_categorical_accuracy: 0.0140 - f

Consider rewriting this model with the Functional API.


5/5 [==============================] - 2s 287ms/step - factorized_top_k/top_1_categorical_accuracy: 9.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0083 - factorized_top_k/top_10_categorical_accuracy: 0.0186 - factorized_top_k/top_50_categorical_accuracy: 0.1099 - factorized_top_k/top_100_categorical_accuracy: 0.2164 - loss: 31035.5990 - regularization_loss: 0.0000e+00 - total_loss: 31035.5990
Top-100 accuracy (train): 0.29.
Top-100 accuracy (test): 0.22.


In [28]:
model = MovielensModel(use_occupations=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


40/40 [==============================] - 8s 158ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0088 - factorized_top_k/top_5_categorical_accuracy: 0.0190 - factorized_top_k/top_10_categorical_accuracy: 0.0280 - factorized_top_k/top_50_categorical_accuracy: 0.0844 - factorized_top_k/top_100_categorical_accuracy: 0.1480 - loss: 14647.0248 - regularization_loss: 0.0000e+00 - total_loss: 14647.0248
Epoch 2/3
40/40 [==============================] - 7s 163ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0032 - factorized_top_k/top_5_categorical_accuracy: 0.0153 - factorized_top_k/top_10_categorical_accuracy: 0.0283 - factorized_top_k/top_50_categorical_accuracy: 0.1176 - factorized_top_k/top_100_categorical_accuracy: 0.2186 - loss: 14178.6772 - regularization_loss: 0.0000e+00 - total_loss: 14178.6772
Epoch 3/3
40/40 [==============================] - 8s 177ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0019 - factorized_top_k/top_5_categorical_accuracy: 0.0142 - f

Consider rewriting this model with the Functional API.


5/5 [==============================] - 1s 287ms/step - factorized_top_k/top_1_categorical_accuracy: 9.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0091 - factorized_top_k/top_10_categorical_accuracy: 0.0205 - factorized_top_k/top_50_categorical_accuracy: 0.1164 - factorized_top_k/top_100_categorical_accuracy: 0.2194 - loss: 31027.8005 - regularization_loss: 0.0000e+00 - total_loss: 31027.8005
Top-100 accuracy (train): 0.30.
Top-100 accuracy (test): 0.22.


Investigate time impact

In [35]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "timestamp": x["timestamp"],
})
movies = movies.map(lambda x: x["movie_title"])

In [36]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))

In [37]:
class UserModel(tf.keras.Model):

  def __init__(self, use_timestamps):
    super().__init__()

    self._use_timestamps = use_timestamps

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_timestamps:
      self.timestamp_embedding = tf.keras.Sequential([
          tf.keras.layers.Discretization(timestamp_buckets.tolist()),
          tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
      ])
      self.normalized_timestamp = tf.keras.layers.Normalization(
          axis=None
      )

      self.normalized_timestamp.adapt(timestamps)

  def call(self, inputs):
    if not self._use_timestamps:
      return self.user_embedding(inputs["user_id"])

    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
    ], axis=1)

In [38]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(movies)

  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)

In [39]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, use_timestamps):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      MovieModel(),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "timestamp": features["timestamp"],
    })
    movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(query_embeddings, movie_embeddings)

In [40]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [41]:
model = MovielensModel(use_timestamps=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


40/40 [==============================] - 10s 197ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0089 - factorized_top_k/top_5_categorical_accuracy: 0.0206 - factorized_top_k/top_10_categorical_accuracy: 0.0312 - factorized_top_k/top_50_categorical_accuracy: 0.0941 - factorized_top_k/top_100_categorical_accuracy: 0.1611 - loss: 14545.1347 - regularization_loss: 0.0000e+00 - total_loss: 14545.1347
Epoch 2/3
40/40 [==============================] - 9s 197ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0036 - factorized_top_k/top_5_categorical_accuracy: 0.0180 - factorized_top_k/top_10_categorical_accuracy: 0.0335 - factorized_top_k/top_50_categorical_accuracy: 0.1400 - factorized_top_k/top_100_categorical_accuracy: 0.2566 - loss: 13948.6204 - regularization_loss: 0.0000e+00 - total_loss: 13948.6204
Epoch 3/3
40/40 [==============================] - 9s 200ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0025 - factorized_top_k/top_5_categorical_accuracy: 0.0198 - 

Consider rewriting this model with the Functional API.


5/5 [==============================] - 3s 352ms/step - factorized_top_k/top_1_categorical_accuracy: 9.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0099 - factorized_top_k/top_10_categorical_accuracy: 0.0218 - factorized_top_k/top_50_categorical_accuracy: 0.1233 - factorized_top_k/top_100_categorical_accuracy: 0.2392 - loss: 30681.9876 - regularization_loss: 0.0000e+00 - total_loss: 30681.9876
Top-100 accuracy (train): 0.36.
Top-100 accuracy (test): 0.24.
